## Reddit Bar Chart Race

In [16]:
from psaw import PushshiftAPI
from collections import Counter
from datetime import timedelta, date
import datetime as dt
import pandas as pd
import bar_chart_race as bcr
from IPython.display import HTML

api = PushshiftAPI()
stopWords = ['to', 'the', 'a', 'to', 'is', 'and', 'in', 'of', 'that', 'it', 'be','at', 'this', 'are', 'be', 'for', 'will', 
             'with', 'have','on', 'by', '|', 'was', 'has', 'after', 'A', 'as', 'from', 'over', 'who', 'says', 'its',"it's"
            'how', 'his', 'her', 'de', '14:', 'vs', 'new', '-', '–', ':', '85', '$', 'us', 'man', 'not', 'you', 'he', 'up', 
            'more', 'out', 'news', 'you', '@rokambola', 'about','first', 'amid', 'an', 'coronavirus:', 'how', 'your', 'than', 
             'what', 'edited:', 'post', 'all', 'against', 'during', '2020', '(lans)', 'we', 'but'] #words to exclude 
#Title words must contain these letters. Words with characters of other languages are excluded
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

### Defining time parameters

In [3]:
StartDate = date(2020, 1, 1)   # start date
EndDate = date(2020, 12, 8)   # end date
delta = EndDate - StartDate       # as timedelta

dates = []
for i in range(0,delta.days):
    day = StartDate + timedelta(days=i)
    dates.append((day.strftime("%m-%d-%Y")))

### Web scraping and sorting

In [5]:
def TitleScrape(numdays):
    big_results = []
    start_epoch=(dt.datetime(2020,12,10)- dt.timedelta(days=numdays+1))   
    end_epoch=(dt.datetime(2020,12,8)- dt.timedelta(days=numdays))      
    results = list(api.search_submissions(after=start_epoch, before=end_epoch, subreddit='news', filter=['title'], limit=1000))
    for i in range(len(results)):
        slicedWords = results[i].title.split()
        for j in slicedWords:
            if j.lower() not in stopWords and j.lower()[0] in alphabet:
                big_results.append(j.title())   #append word with capitlaized first letter if word not in stopwords
    return big_results

In [6]:
AllResults = []
for i in range(len(dates)):    #specifcy how many days ago you want scrape to start
    freqDist = Counter(TitleScrape(i))
    for k,v in list(freqDist.items()):
        if v<3:
            del freqDist[k]   
    AllResults.append(freqDist)

### Creating bar chart race where bars are total word sums

In [8]:
df = pd.DataFrame([AllResults[i] for i in range(len(AllResults))], dtype=object).fillna(0).astype(int)
df = df.iloc[::-1]  
df = df.cumsum()
df.insert(loc=0, column='Day Column', value=dates)
df = df.set_index('Day Column')

In [ ]:
bcr.bar_chart_race(df=df, filename='news_days_vis5.mp4', title = "Cumulative Word Appearances in 2020 r/news Titles", n_bars=10, steps_per_period=10, period_length=400)

### Creating bar chart race where bars are 75-day moving sums

In [12]:
dfRolling = pd.DataFrame([AllResults[i] for i in range(len(AllResults))], dtype=object).fillna(0).astype(int)
dfRolling = dfRolling.iloc[::-1]
dfRolling = dfRolling.rolling(min_periods=1, window=75).sum()
dfRolling.insert(loc=0, column='Day Column', value=dates)
dfRolling = dfRolling.set_index('Day Column')

In [ ]:
bcr.bar_chart_race(df=dfRolling, filename='news_rolling3.mp4', title = "Moving Total of Word Appearances in 2020 r/news Titles", n_bars=10, steps_per_period=10, period_length=400)

link to video: https://www.youtube.com/watch?v=MkMVSotK8mk